#### evaluation

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Layer
from tensorflow.keras.layers import Reshape, Concatenate

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import tensorflow.keras.backend as K

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.attention_weights = None  # Save attention weights here

    def call(self, inputs, training, return_attention=False):
        attn_output, attn_weights = self.att(inputs, inputs, return_attention_scores=True)
        self.attention_weights = attn_weights  # Save attention weights
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        if return_attention:
            return self.layernorm2(out1 + ffn_output), attn_weights
        return self.layernorm2(out1 + ffn_output)
        
def build_cascade_transformer_model_with_fingerprint(input_shape, fingerprint_shape, num_heads=4, ff_dim=128, task1_classes=2, task2_classes=2):
    # Primary input for sequence data
    sequence_input = Input(shape=input_shape, name="sequence_input")  # Shape: (sequence_length, feature_dim)

    # Fingerprint input
    fingerprint_input = Input(shape=(fingerprint_shape,), name="fingerprint_input")  # Shape: (fingerprint_dim,)
    fingerprint_dense = Dense(input_shape[1], activation="relu", name="fingerprint_dense")(fingerprint_input)  # Match feature_dim
    fingerprint_expanded = Reshape((1, input_shape[1]), name="fingerprint_expanded")(fingerprint_dense)  # Shape: (1, feature_dim)

    # Combine sequence input and fingerprint input along the sequence dimension
    combined_input = Concatenate(axis=1, name="combined_input")([sequence_input, fingerprint_expanded])  # Shape: (sequence_length + 1, feature_dim)

    # First Transformer Block
    x = TransformerBlock(embed_dim=input_shape[1], num_heads=num_heads, ff_dim=ff_dim)(combined_input)
    x = TransformerBlock(embed_dim=input_shape[1], num_heads=num_heads, ff_dim=ff_dim)(x)
    x = Flatten()(x)
    shared_dense = Dense(input_shape[1], activation='relu', name="shared_dense_1")(x)
    shared_dense_task1 = Dense(128, activation='relu', name="shared_dense_2")(shared_dense)
    shared_dense_task1 = Dense(32, activation='relu', name="shared_dense_3")(shared_dense_task1)
    shared_dropout = Dropout(0.3, name="shared_dropout_1")(shared_dense_task1)

    # Task 1 Output
    task1_output = Dense(task1_classes, activation='softmax', name="task1_output")(shared_dropout)

    # Cascade Task 1 Output to Task 2 Input
    task1_features = Dense(input_shape[1], activation='relu', name="task1_features")(task1_output)
    task1_features_expanded = tf.expand_dims(task1_features, axis=1)

    # Expand shared_dense and concatenate with task1_features
    shared_dense_expanded = tf.expand_dims(shared_dense, axis=1)
    cascade_input = tf.concat([shared_dense_expanded, task1_features_expanded], axis=1)  # Shape: (batch_size, 2, input_shape[1])

    # Task 2 Transformer Block
    task2_x = TransformerBlock(embed_dim=input_shape[1], num_heads=num_heads, ff_dim=ff_dim)(cascade_input)
    task2_x = Flatten()(task2_x)
    task2_x = Dense(128, activation='relu', name="task2_dense_1")(task2_x)
    task2_x = Dense(32, activation='relu', name="task2_dense_2")(task2_x)

    # Task 2 Output
    task2_output = Dense(task2_classes, activation='softmax', name="task2_output")(task2_x)

    # Build Model
    model = Model(inputs=[sequence_input, fingerprint_input], outputs=[task1_output, task2_output], name="cascade_transformer_with_fingerprint")
    return model

In [2]:
test_inputs = np.load('data/test_inputs.npy')
test_fingerprints = np.load('data/test_fingerprints.npy')
test_inputs = test_inputs.astype('float32')
test_fingerprints = test_fingerprints.astype('float32')

In [4]:
# Load the trained model
model = build_cascade_transformer_model_with_fingerprint(
    input_shape=(1024, 80),
    fingerprint_shape=881
)
model.load_weights('../../model/cascade_transformer_model_weights.h5')

In [5]:
test_predictions = model.predict([test_inputs, test_fingerprints])
print(test_predictions)

[array([[9.9287346e-30, 1.0000000e+00],
       [1.8304142e-29, 1.0000000e+00],
       [1.3748761e-29, 1.0000000e+00]], dtype=float32), array([[0.47746563, 0.5225344 ],
       [0.47191617, 0.52808386],
       [0.47424385, 0.5257562 ]], dtype=float32)]
